In [ ]:
# View pull request status
pr = repo.get_pull(pr.number)
print(f'PR #{pr.number} status: {pr.state}')
print(f'Merged: {pr.is_merged()}')
print(f'URL: {pr.html_url}')


In [ ]:
# Create a pull request using PyGithub
pr = repo.create_pull(
    title='Colab Demo PR',
    body='This PR was created from Colab.',
    head=NEW_BRANCH,
    base='master'  # Change if needed
)
print(f'Pull request created: {pr.html_url}')


In [ ]:
# Commit and push changes
# Example: create a dummy file and commit it
with open('colab_demo.txt', 'w') as f:
    f.write('This file was created in Colab!')

subprocess.run(['git', 'add', 'colab_demo.txt'])
subprocess.run(['git', 'commit', '-m', 'Add colab_demo.txt from Colab'])
subprocess.run(['git', 'push', '--set-upstream', 'origin', NEW_BRANCH])
print('Committed and pushed changes to remote branch.')


In [ ]:
# Create a new branch using git
import shutil

NEW_BRANCH = 'colab-demo-branch'
os.chdir(REPO_DIR)

# Create and checkout new branch
def branch_exists(branch):
    result = subprocess.run(['git', 'branch', '--list', branch], capture_output=True, text=True)
    return branch in result.stdout

if not branch_exists(NEW_BRANCH):
    subprocess.run(['git', 'checkout', '-b', NEW_BRANCH])
    print(f'Created and switched to branch: {NEW_BRANCH}')
else:
    subprocess.run(['git', 'checkout', NEW_BRANCH])
    print(f'Switched to existing branch: {NEW_BRANCH}')


In [ ]:
# List branches in the repository using GitHub API
repo = g.get_repo('Solventerritory/FarmFederate-Advisor')
branches = repo.get_branches()
print('Branches:')
for branch in branches:
    print(branch.name)


In [ ]:
# Clone a repository using git
import os
import subprocess

REPO_URL = 'https://github.com/Solventerritory/FarmFederate-Advisor.git'
BRANCH = 'master'  # Change if needed
REPO_DIR = 'FarmFederate-Advisor'

if not os.path.exists(REPO_DIR):
    subprocess.run(['git', 'clone', '-b', BRANCH, REPO_URL])
else:
    print(f'Repository already cloned at {REPO_DIR}')


In [ ]:
# Authenticate with GitHub using a personal access token
from getpass import getpass
from github import Github

GITHUB_TOKEN = getpass('Enter your GitHub Personal Access Token: ')
g = Github(GITHUB_TOKEN)
user = g.get_user()
print(f'Authenticated as: {user.login}')


In [ ]:
# Install required libraries
!pip install PyGithub


# GitHub Operations in Colab
This notebook demonstrates how to perform common GitHub operations using Python and Colab. You will:
- Authenticate with GitHub
- Clone a repository
- List branches
- Create a new branch
- Commit and push changes
- Create a pull request
- View pull request status

**Requirements:**
- A GitHub personal access token (PAT) with repo permissions
- The `PyGithub` library (installed below)


In [ ]:
# STEP 1: Keep-Alive & Protection (RUN FIRST!)
from IPython.display import Javascript, display
import time

# Simple keep-alive - clicks connect every 60 seconds
js_code = """
setInterval(function() {
  var btn = document.querySelector('colab-toolbar-button#connect');
  if (btn) {
    btn.click();
    console.log('Keeping alive:', new Date().toLocaleTimeString());
  }
}, 60000);
"""

display(Javascript(js_code))
print("✅ Keep-alive enabled!")
print("⚠️ Keep this browser tab open (can be in background)")
print(f"Started at: {time.strftime('%H:%M:%S')}")

In [ ]:
# STEP 2: GPU Check & Memory Management
import torch
import gc
import os

# Memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True,max_split_size_mb:512'

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
    
    # Set memory limit
    torch.cuda.set_per_process_memory_fraction(0.85)
    print(f"   Limit: {gpu_memory * 0.85:.1f} GB (85%)")
else:
    raise RuntimeError("❌ NO GPU! Go to Runtime → Change runtime type → GPU")

# Memory clearing function
def clear_gpu():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

print("✅ Memory management ready")

In [ ]:
# STEP 3: Mount Google Drive (IMPORTANT!)
from google.colab import drive

drive.mount('/content/drive')

# Create results directory
import os
results_dir = '/content/drive/MyDrive/FarmFederate_Results'
os.makedirs(results_dir, exist_ok=True)
os.makedirs(f'{results_dir}/checkpoints', exist_ok=True)
os.makedirs(f'{results_dir}/results', exist_ok=True)
os.makedirs(f'{results_dir}/plots', exist_ok=True)

os.environ['DRIVE_RESULTS_DIR'] = results_dir

print(f"✅ Google Drive mounted")
print(f"   Results will save to: {results_dir}")
print("   ⚠️ This prevents data loss if disconnected!")

In [ ]:
# STEP 4: Install Dependencies
%%capture
!pip install -q transformers datasets peft accelerate evaluate scikit-learn
!pip install -q sentencepiece protobuf timm torch torchvision
!pip install -q matplotlib seaborn pandas pillow

print("✅ Dependencies installed")

In [ ]:
# STEP 5: Clone Repository
import os

if not os.path.exists('/content/FarmFederate-Advisor'):
    !git clone https://github.com/Solventerritory/FarmFederate-Advisor.git
    print("✅ Repository cloned")
else:
    !cd /content/FarmFederate-Advisor && git pull
    print("✅ Repository updated")

os.chdir('/content/FarmFederate-Advisor/backend')
print(f"Working directory: {os.getcwd()}")

In [ ]:
# STEP 6: Auto-Configure for GPU
import sys
sys.path.insert(0, '/content/FarmFederate-Advisor/backend')

# Auto-detect GPU and set batch size
if gpu_memory < 16:
    batch_size = 2
    lora_rank = 4
    print("📊 T4 Configuration (Conservative)")
elif gpu_memory < 24:
    batch_size = 4
    lora_rank = 8
    print("📊 V100 Configuration")
else:
    batch_size = 8
    lora_rank = 16
    print("📊 A100 Configuration")

print(f"   Batch size: {batch_size}")
print(f"   LoRA rank: {lora_rank}")

os.environ['COLAB_GPU'] = '1'
os.environ['COLAB_BATCH_SIZE'] = str(batch_size)
os.environ['COLAB_LORA_RANK'] = str(lora_rank)

print("✅ Configuration complete")

In [ ]:
# STEP 7: Run Training
import time
print("🚀 Starting training...")
print(f"   Time: {time.strftime('%H:%M:%S')}")
print("   Estimated duration: 3-5 hours (T4), 2-3 hours (V100)")
print("\n⏳ Training in progress...\n")

start = time.time()

try:
    from federated_complete_training import main
    main()
    
    elapsed = (time.time() - start) / 3600
    print(f"\n✅ COMPLETE! Time: {elapsed:.2f} hours")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

finally:
    # Backup to Drive
    drive_dir = os.environ.get('DRIVE_RESULTS_DIR')
    if drive_dir:
        print("\n💾 Backing up to Google Drive...")
        !cp -r ../results/* {drive_dir}/results/ 2>/dev/null || true
        !cp -r ../plots/* {drive_dir}/plots/ 2>/dev/null || true
        print("✅ Backup complete")
    
    clear_gpu()

In [ ]:
# STEP 8: Generate Plots
clear_gpu()

print("📊 Generating plots...")
!python comprehensive_plotting.py

# Copy to Drive
drive_dir = os.environ.get('DRIVE_RESULTS_DIR')
if drive_dir:
    !cp -r ../plots/* {drive_dir}/plots/ 2>/dev/null || true

print("✅ Plots ready")

In [ ]:
# STEP 9: View Results
import json
from IPython.display import Image, display

results_file = '../results/all_results.json'
if os.path.exists(results_file):
    with open(results_file, 'r') as f:
        results = json.load(f)
    
    sorted_results = sorted(results, key=lambda x: x.get('final_metrics', {}).get('f1_macro', 0), reverse=True)
    
    print("="*60)
    print("🏆 TOP 10 MODELS")
    print("="*60)
    
    for i, model in enumerate(sorted_results[:10], 1):
        name = model.get('config', {}).get('name', 'Unknown')
        metrics = model.get('final_metrics', {})
        f1 = metrics.get('f1_macro', 0)
        acc = metrics.get('accuracy', 0)
        print(f"{i:2d}. {name:30s} F1: {f1:.4f} | Acc: {acc:.4f}")
    
    # Display plots
    print("\n📈 Plots:")
    plots = [f for f in os.listdir('../plots') if f.endswith('.png')][:3]
    for plot in plots:
        print(f"   - {plot}")
        display(Image(f'../plots/{plot}', width=700))
else:
    print("⚠️ No results found")

print(f"\n💾 All results saved to: {os.environ.get('DRIVE_RESULTS_DIR')}")

In [ ]:
# STEP 10: Download Results (Optional)
from google.colab import files
import shutil

print("📦 Creating download packages...")

if os.path.exists('../results'):
    shutil.make_archive('/content/results', 'zip', '../results')
    print("✅ results.zip ready")

if os.path.exists('../plots'):
    shutil.make_archive('/content/plots', 'zip', '../plots')
    print("✅ plots.zip ready")

print("\n📥 Downloading...")
if os.path.exists('/content/results.zip'):
    files.download('/content/results.zip')
if os.path.exists('/content/plots.zip'):
    files.download('/content/plots.zip')

print("\n✅ TRAINING COMPLETE!")
print("📊 Results in: Google Drive + Downloads folder")